### Limpeza da base de dados de estoque 



### Descrição das Colunas - Base de Estoque

| Nome da Coluna           | Descrição                                                                                  | Exemplo                                                                 |
|--------------------------|-------------------------------------------------------------------------------------------|-------------------------------------------------------------------------|
| **Código**               | Identificador único do produto no sistema.                                                | `1`, `3`, `5`                                                          |
| **Produto**              | Nome/descrição do produto comercializado.                                                 | `"LAPIS Nº2 HB PT C/144 SERELEPE ECOLE"`, `"ACUCAR COMUM"`             |
| **Embalagem**            | Tipo de embalagem do produto (formato e quantidade por embalagem).                        | `"CX/1"` (Caixa com 1 unidade), `"KG/1"` (Kilograma unitário)          |
| **Unidade**              | Unidade de medida primária para venda ou controle.                                        | `"CX"` (Caixa), `"UN"` (Unidade), `"KG"` (Kilograma)                   |
| **Código da Marca**      | Identificador único da marca/fabricante do produto.                                       | `1` (Maria Geni), `4281` (Grupo Master)                                |
| **Marca**                | Nome da marca ou fornecedor do produto.                                                   | `"COMERCIO E IMPORTACAO SERTIC LTDA"`                           |
| **Quantidade estoque**   | Quantidade disponível em estoque (vazia indica registro sem dados).                       | `0`, `61`                                                              |
| **Preço de custo**       | Preço unitário de custo do produto (formato numérico).                                    | `9`, `1.48`, `2.5`                                                     |
| **Código da categoria**  | Identificador numérico da categoria do produto.                                           | `708` (Lápis), `199` (Açúcar)                                          |
| **Categoria**            | Nome da categoria do produto, com abreviação entre parênteses.                            | `"LAPIS ( LAP )"`,                                          |

In [ ]:
import pandas as pd

In [ ]:
df_estoque = pd.read_excel("bases/relatorio_produtos.xlsx")

### Propriedades da base

In [ ]:
print(f"Formato dos dados {df_estoque.shape}")

In [ ]:
df_estoque.columns.to_list()

In [ ]:
df_estoque.head()

In [ ]:
df_estoque.dtypes

In [ ]:
df_estoque.describe()

In [ ]:
print("Quantidade valores Nulos")
df_estoque.isnull().sum()

In [ ]:
print("Quantidade de valores zerados")
colunas_numericas = df_estoque.select_dtypes(include='number')
for rotulo in colunas_numericas:
    qt_zeros = (colunas_numericas[rotulo] == 0).sum()
    print(f"\n{rotulo} contém {qt_zeros} zeros")

In [ ]:
print("Quantidade de valores negativos")
for rotulo in colunas_numericas:
    qt_negativos = (colunas_numericas[rotulo] < 0).sum()
    print(f"\n{rotulo} contém {qt_negativos} negativos")

### Tratamento inicial 

In [ ]:
# Na coluna Quantidade estoque contém alguns valores nulos e negativos, estes serão convertidos para '0'.
df_estoque.loc[df_estoque["Quantidade estoque"] < 0, "Quantidade estoque"] = 0
df_estoque['Quantidade estoque'] = df_estoque['Quantidade estoque'].fillna(0)

In [ ]:
# Separar somente as colunas que serão usadas no modelo
df_util_estoque = df_estoque[['Código', 'Produto', 'Código da categoria', 'Categoria', 'Código da Marca', 'Marca', 'Preço de custo', 'Quantidade estoque']]
df_estoque.head()

#### Para garantir a integridade dos dados e a eficácia do sistema de recomendação/cross-selling, é essencial verificar:

#### Unicidade dos Identificadores

Código: Cada valor deve ser único e corresponder a um único Produto.

Código da Marca: Deve ser único para cada Marca registrada.

Código da categoria: Deve ser único e vinculado a uma única Categoria.

In [ ]:
df_util_estoque.nunique()

#### Garantindo a integridade dos códigos e seus respectivas descrições

In [ ]:
codigos_produtos = df_util_estoque['Código'].nunique()
produtos_unicos = df_util_estoque['Produto'].nunique()
print(f"{codigos_produtos} Código de Produtos Únicos")
print(f"{produtos_unicos} Produtos Únicos")

In [ ]:
# Verificando se há códigos duplicados associados a produtos diferentes
codigos_produtos_duplicados = df_util_estoque[df_util_estoque["Código"].duplicated(keep=False)]
resultado = codigos_produtos_duplicados.sort_values("Código")
if resultado.isnull:
    print("Não exitem códigos de produtos duplicados")
else:
    print("Existem códigos duplicados, erro crítico")

In [ ]:
# Remover todos os produtos cuja descrição contém mais de um código 
produto_multiplos_codigos = df_util_estoque.groupby("Produto")["Código"].nunique()
produtos_problemas = produto_multiplos_codigos[produto_multiplos_codigos > 1].index
df_util_estoque = df_util_estoque[~df_util_estoque["Produto"].isin(produtos_problemas)]

#### Garantindo a integridade dos códigos e suas respectivas categorias

In [ ]:
codigos_categoria = df_util_estoque['Código da categoria'].nunique()
categorias_unicas = df_util_estoque['Categoria'].nunique()
print(f"{codigos_categoria} Código de Categoria Únicos")
print(f"{categorias_unicas} Categoria Únicos")

In [ ]:
# Verificando se há códigos duplicados associados a categorias diferentes
codigos_categorias_duplicados = df_util_estoque[df_util_estoque["Código da categoria"].duplicated(keep=False)]
resultado = codigos_categorias_duplicados.sort_values("Código da categoria")
if resultado.isnull:
    print("Não exitem códigos de Categorias duplicadas")
else:
    print("Existem códigos duplicados, erro crítico")

In [ ]:
# Remover todos as categorias cuja descrição contém mais de um código 
categoria_multiplos_codigos = df_util_estoque.groupby("Categoria")["Código da categoria"].nunique()
categorias_problemas = categoria_multiplos_codigos[categoria_multiplos_codigos > 1].index
df_util_estoque = df_util_estoque[~df_util_estoque["Categoria"].isin(categorias_problemas)]

#### Garantindo a integridade dos códigos e suas respectivas marcas

In [ ]:
codigos_marca = df_util_estoque['Código da Marca'].nunique()
marcas_unicas = df_util_estoque['Marca'].nunique()
print(f"{codigos_marca} Código de marcas Únicos")
print(f"{marcas_unicas} Marcas Únicas")

In [ ]:
# Verificando se há códigos duplicados associados a categorias diferentes
codigos_marcas_duplicados = df_util_estoque[df_util_estoque["Código da Marca"].duplicated(keep=False)]
resultado = codigos_marcas_duplicados.sort_values("Código da Marca")
if resultado.isnull:
    print("Não exitem códigos de Marcas duplicados")
else:
    print("Existem códigos duplicados, erro crítico")

In [ ]:
# Remover todos as marcas cuja descrição contém mais de um código 
categoria_multiplos_codigos = df_util_estoque.groupby("Marca")["Código da Marca"].nunique()
categorias_problemas = categoria_multiplos_codigos[categoria_multiplos_codigos > 1].index
df_util_estoque = df_util_estoque[~df_util_estoque["Marca"].isin(categorias_problemas)]

##### Balanço das trataivas

In [ ]:
df_util_estoque.nunique()

In [ ]:
print("Diferença entre códigos e descrições")
print(f"\nProduto: {codigos_produtos-produtos_unicos} formam excluidos {((codigos_produtos-produtos_unicos)*100 / codigos_produtos):.2f} %")
print(f"\nCategoria: {codigos_categoria-categorias_unicas} formam excluidos {((codigos_categoria-categorias_unicas)*100 / codigos_categoria):.2f} %")
print(f"\nMarca: {codigos_marca-marcas_unicas} formam excluidos {((codigos_marca-marcas_unicas)*100 / codigos_marca):.2f} %")
print(f"\nTotal base original {df_estoque.shape[0]}\nTotal base sem duplicatas {df_util_estoque.shape[0]}")
print(f"um exclusão de {((df_estoque.shape[0] - df_util_estoque.shape[0]) * 100 / df_estoque.shape[0]):.2f} % de registros duplicados")

In [ ]:
def verificar_limpeza_base(df, coluna_codigo='Código', coluna_produto='Produto'):
    """
    Verifica se a base tratada ainda precisa de ajustes.
    
    Retorna:
        Um relatório com problemas residuais.
    """
    # 1. Verifica duplicatas em combinação de código + produto (chave única)
    duplicatas_chave = df.duplicated(subset=[coluna_codigo, coluna_produto], keep=False)
    if duplicatas_chave.any():
        print(f"⚠️ **Atenção**: {duplicatas_chave.sum()} registros duplicados em 'Código + Produto'.")
        display(df[duplicatas_chave].sort_values(coluna_produto).head())
    else:
        print("✅ Sem duplicatas em 'Código + Produto'.")

    # 2. Verifica valores nulos em colunas críticas
    colunas_criticas = [coluna_codigo, coluna_produto, 'Categoria', 'Marca', 'Quantidade estoque']
    nulos = df[colunas_criticas].isnull().sum()
    if nulos.any():
        print("\n⚠️ **Valores nulos encontrados**:")
        display(nulos[nulos > 0])
    else:
        print("\n✅ Sem valores nulos em colunas críticas.")

    # 3. Verifica valores zerados/negativos no estoque
    estoque_zerado = (df['Quantidade estoque'] == 0).sum()
    estoque_negativo = (df['Quantidade estoque'] < 0).sum()
    if estoque_zerado > 0 or estoque_negativo > 0:
        print(f"\n⚠️ **Problemas no estoque**: {estoque_zerado} zerados e {estoque_negativo} negativos.")
    else:
        print("\n✅ Estoque sem valores zerados/negativos.")

    # 4. Verifica inconsistências entre código e produto
    produtos_por_codigo = df.groupby(coluna_codigo)[coluna_produto].nunique()
    codigos_problema = produtos_por_codigo[produtos_por_codigo > 1]
    if not codigos_problema.empty:
        print(f"\n⚠️ **Inconsistências**: {len(codigos_problema)} códigos com múltiplos produtos.")
        display(codigos_problema.head())
    else:
        print("\n✅ Todos os códigos têm apenas 1 produto associado.")

    # 5. Estatísticas finais
    print("\n📊 **Resumo final**:")
    print(f"- Registros totais: {len(df)}")
    print(f"- Produtos únicos: {df[coluna_produto].nunique()}")
    print(f"- Códigos únicos: {df[coluna_codigo].nunique()}")

# Uso:
verificar_limpeza_base(df_util_estoque)

In [ ]:
df_util_estoque.dtypes

In [ ]:
df_util_estoque = df_util_estoque.astype({
    'Quantidade estoque' : 'int64'
})
df_util_estoque["Preço de custo"] = df_util_estoque["Preço de custo"].round(2)

In [ ]:
df_util_estoque.rename(columns = {"Código" : "Código produto"}, inplace=True)

In [ ]:
df_util_estoque.to_excel("bases/bases_limpas/base_estoque_limpo.xlsx", index=False)